In [26]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from torchsummary import summary
from sklearn.model_selection import train_test_split

데이터 로드 및 확인

In [7]:
data = pd.read_csv('diabetes.csv')  # CSV 파일 경로
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


데이터 엔코딩

In [14]:
# 예시 데이터프레임 (데모용 데이터)
data = pd.DataFrame({
    'Pregnancies': ['1', '2', '3'],
    'Glucose': ['85', '90', '95'],
    'BloodPressure': ['70', '80', '75'],
    'SkinThickness': ['20', '25', '23'],
    'Insulin': ['120', '130', '125'],
    'BMI': ['30.5', '32.0', '33.1'],
    'DiabetesPedigreeFunction': ['0.5', '0.6', '0.4'],
    'Age': ['25', '30', '35'],
    'Outcome': ['0', '1', '0']
})

# 열 이름을 사용자가 제공한 대로 설정
data.columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Label Encoding을 적용할 열 목록
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']

# LabelEncoder 객체를 저장할 딕셔너리 초기화
label_encoders = {}

# 각 열에 대해 LabelEncoder를 적용
for column in columns:
    label_encoders[column] = LabelEncoder()  # LabelEncoder 객체 생성
    data[column] = label_encoders[column].fit_transform(data[column])  # 해당 열에 Label Encoding 적용

# 변환된 데이터프레임 출력
print(data)

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin  BMI  \
0            0        0              0              0        0    0   
1            1        1              2              2        2    1   
2            2        2              1              1        1    2   

   DiabetesPedigreeFunction  Age  Outcome  
0                         1    0        0  
1                         2    1        1  
2                         0    2        0  


레이블 및 전처리

In [17]:
# 데이터와 타겟 분리
X = data.drop('Outcome', axis=1).values
y = data['Outcome'].values

데이터 스케일링

In [23]:
# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

훈련 데이터

In [27]:
# 훈련/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

PyTorch 텐서로 데이터 변환

In [30]:
# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)  # X_train을 float32 타입의 PyTorch 텐서로 변환
y_train = torch.tensor(y_train, dtype=torch.int64)    # y_train을 int64 타입의 PyTorch 텐서로 변환
X_test = torch.tensor(X_test, dtype=torch.float32)    # X_test를 float32 타입의 PyTorch 텐서로 변환
y_test = torch.tensor(y_test, dtype=torch.int64)      # y_test를 int64 타입의 PyTorch 텐서로 변환

C:\Users\Admin\AppData\Local\Temp\ipykernel_4516\4017085670.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train, dtype=torch.float32)  # X_train을 float32 타입의 PyTorch 텐서로 변환
C:\Users\Admin\AppData\Local\Temp\ipykernel_4516\4017085670.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.int64)    # y_train을 int64 타입의 PyTorch 텐서로 변환
C:\Users\Admin\AppData\Local\Temp\ipykernel_4516\4017085670.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_te

데이터 로드

In [31]:
# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)  # 학습 데이터를 TensorDataset 형태로 변환
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # 학습용 DataLoader 생성

test_dataset = TensorDataset(X_test, y_test)  # 테스트 데이터를 TensorDataset 형태로 변환
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)  # 테스트용 DataLoader 생성

# 데이터 크기 확인
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([2, 8]), torch.Size([1, 8]), torch.Size([2]), torch.Size([1]))

모델 정의

In [43]:
class DiabetesDense(nn.Module):
    def __init__(self):
        super(DiabetesDense, self).__init__()
        # 8개의 입력 특성 (Diabetes 데이터셋에 맞게 수정)
        self.fc1 = nn.Linear(8, 64)  # 첫 번째 은닉층, 8개의 입력을 받아 64개의 뉴런으로 처리
        self.fc2 = nn.Linear(64, 32)  # 두 번째 은닉층, 64개의 뉴런을 받아 32개의 뉴런으로 처리
        self.fc3 = nn.Linear(32, 1)   # 출력층, 1개의 뉴런 (이진 분류)
        self.sigmoid = nn.Sigmoid()   # 시그모이드 활성화 함수 (이진 분류 확률 출력)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))  # ReLU 활성화 함수 (첫 번째 은닉층)
        x = torch.relu(self.fc2(x))  # ReLU 활성화 함수 (두 번째 은닉층)
        x = self.fc3(x)              # 출력층
        x = self.sigmoid(x)          # 시그모이드 함수로 이진 분류 확률 계산
        return x

# 모델 초기화
model = DiabetesDense()

# 모델, 손실 함수, 옵티마이저 설정
model = DiabetesModel()
criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss는 정수형 라벨을 사용합니다.
optimizer = optim.Adam(model.parameters(), lr=0.001)


손실 함수 및 최적화 기법 정의

In [35]:
# 손실 함수 설정
criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류 문제에서 사용하는 교차 엔트로피 손실 함수

# 옵티마이저 설정
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 옵티마이저를 사용하여 모델의 파라미터를 최적화, 학습률(lr)은 0.001로 설정


모델 학습

In [44]:
# 필요한 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim

# Variables to store loss and accuracy
train_losses = []  # 학습 손실을 저장할 리스트
test_accuracies = []  # 테스트 정확도를 저장할 리스트

# Training loop
num_epochs = 20  # 학습할 에폭 수
for epoch in range(num_epochs):  # 각 에폭에 대해 반복
    model.train()  # 모델을 훈련 모드로 설정
    running_loss = 0.0  # 각 에폭에서 손실을 누적할 변수
    for inputs, labels in train_dataloader:  # 훈련 데이터에서 배치를 하나씩 꺼내옴
        # Zero the parameter gradients (기울기 초기화)
        optimizer.zero_grad()

        # Forward pass (순전파)
        outputs = model(inputs)  # 모델에 입력값을 넣고 예측값을 얻음
        loss = criterion(outputs, labels)  # 예측값과 실제값 사이의 손실 계산

        # Backward pass and optimize (역전파와 가중치 최적화)
        loss.backward()  # 손실에 대해 역전파 수행
        optimizer.step()  # 옵티마이저로 가중치를 업데이트

        running_loss += loss.item()  # 손실 값을 누적

    # Calculate average loss over an epoch (에폭마다 평균 손실 계산)
    train_losses.append(running_loss / len(train_dataloader))  # 한 에폭의 평균 손실 저장

    # Evaluate on test data (테스트 데이터로 모델 평가)
    model.eval()  # 모델을 평가 모드로 설정
    correct = 0  # 정확도 계산을 위한 변수
    total = 0  # 총 샘플 수
    with torch.no_grad():  # 평가 시에는 기울기를 계산하지 않음
        for inputs, labels in test_dataloader:  # 테스트 데이터에서 배치를 하나씩 꺼냄
            outputs = model(inputs)  # 모델에 입력값을 넣고 예측값을 얻음
            _, predicted = torch.max(outputs.data, 1)  # 예측값 중 가장 큰 값(클래스) 선택
            total += labels.size(0)  # 전체 샘플 수 갱신
            correct += (predicted == labels).sum().item()  # 예측이 맞은 샘플의 수를 누적

    accuracy = 100 * correct / total  # 정확도 계산
    test_accuracies.append(accuracy)  # 테스트 정확도 저장

    # 에폭별 손실과 정확도 출력
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_losses[-1]:.4f}, Accuracy: {accuracy:.2f}%")

print("Training complete.")  # 학습 완료 후 출력


Epoch 1/20, Loss: 0.7190, Accuracy: 0.00%
Epoch 2/20, Loss: 0.6964, Accuracy: 0.00%
Epoch 3/20, Loss: 0.6743, Accuracy: 0.00%
Epoch 4/20, Loss: 0.6537, Accuracy: 0.00%
Epoch 5/20, Loss: 0.6354, Accuracy: 0.00%
Epoch 6/20, Loss: 0.6174, Accuracy: 0.00%
Epoch 7/20, Loss: 0.6002, Accuracy: 0.00%
Epoch 8/20, Loss: 0.5839, Accuracy: 0.00%
Epoch 9/20, Loss: 0.5677, Accuracy: 0.00%
Epoch 10/20, Loss: 0.5516, Accuracy: 0.00%
Epoch 11/20, Loss: 0.5359, Accuracy: 0.00%
Epoch 12/20, Loss: 0.5204, Accuracy: 0.00%
Epoch 13/20, Loss: 0.5052, Accuracy: 0.00%
Epoch 14/20, Loss: 0.4902, Accuracy: 0.00%
Epoch 15/20, Loss: 0.4766, Accuracy: 0.00%
Epoch 16/20, Loss: 0.4635, Accuracy: 0.00%
Epoch 17/20, Loss: 0.4502, Accuracy: 0.00%
Epoch 18/20, Loss: 0.4368, Accuracy: 0.00%
Epoch 19/20, Loss: 0.4236, Accuracy: 0.00%
Epoch 20/20, Loss: 0.4104, Accuracy: 0.00%
Training complete.


모델 평가

In [45]:
# Variables to store evaluation metrics
train_losses = []  # 학습 손실을 저장할 리스트
test_accuracies = []  # 테스트 정확도를 저장할 리스트

# Evaluation
model.eval()  # 모델을 평가 모드로 설정
all_labels = []  # 실제 라벨 값을 저장할 리스트
all_predictions = []  # 예측값을 저장할 리스트
with torch.no_grad():  # 기울기 계산 비활성화
    for inputs, labels in test_dataloader:  # 테스트 데이터 배치마다 반복
        outputs = model(inputs)  # 모델에 입력값을 넣고 예측값 얻음
        _, predicted = torch.max(outputs.data, 1)  # 가장 높은 값의 클래스 예측
        all_labels.extend(labels.cpu().numpy())  # 실제 라벨 리스트에 추가
        all_predictions.extend(predicted.cpu().numpy())  # 예측값 리스트에 추가

# Convert to numpy arrays
all_labels = np.array(all_labels)  # 실제 라벨을 numpy 배열로 변환
all_predictions = np.array(all_predictions)  # 예측값을 numpy 배열로 변환

# Calculate metrics
conf_matrix = confusion_matrix(all_labels, all_predictions)  # 혼동 행렬 계산
f1 = f1_score(all_labels, all_predictions, average='weighted')  # F1 스코어 계산
precision = precision_score(all_labels, all_predictions, average='weighted')  # Precision 계산
recall = recall_score(all_labels, all_predictions, average='weighted')  # Recall 계산

# 출력
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'F1 Score: {f1:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')


Confusion Matrix:
[[0 1]
 [0 0]]
F1 Score: 0.00
Precision: 0.00
Recall: 0.00


c:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
